## Step 4: 수치 미분

머신러닝을 포함한 많은 공학 계열의 기본은 미분이다. 이러한 미분이 무엇인지 간단히 짚어보고 이를 직접 구현해보도록 하자. 

### 4.1 미분이란

미분(differentiation)은 쉽게 말해 '변화율'이다. 어떤 물체의 속도는 그 물체의 시간에 따른 위치 변화율이며 이를 '위치의 미분'이라고 표현할 수 있다. 또 움직이는 물체의 가속도는 시간에 대한 속도의 변화율, 즉 속도의 미분이 된다.

여기서 공통적으로 기준을 두는 것은 바로 시간인데, 미분의 정의상 이 '시간'은 극도로 짧은 시간, 즉 순간을 의미한다. 순간에 대한 특정 대상의 변화율이 바로 미분이며 $f(x)$의 미분 $f'(x)$를 수식으로 나타내면 다음과 같다.

$$
f'(x) = \displaystyle \lim_{h\rightarrow0}{{f(x+h) - f(x)}\over{h}}
$$

이러한 $f'(x)$를 $f(x)$의 도함수라고 한다.

### 4.2 수치 미분 구현

수학적으로 미분을 정의할 때는 $h$를 0에 무한히 근접해지도록 한다. 다만 이러한 것은 컴퓨터에서 구현하기 어려우므로 보통 $h=1e-4$와 같은 '매우 작은 값'으로 대체하곤 한다. 이처럼 아주 작은 값을 수치적으로 정하여 함수의 변화량을 구하는 방법을 '수치 미분(numerical differentiation)'이라고 한다.

물론 이러한 방식은 진정한 미분에 '근접'할뿐 결코 같아질 수 없으므로 오차를 갖는데, 이러한 근사 오차를 줄이는 방법으로 '중앙 차분(centered difference)'이라는 방법을 사용한다. 이는 $f(x+h)-f(x)$가 아닌 $f(x+h)-f(x-h)$를 의미하는데, 수치 미분에서는 전진 차분인 $f(x+h)-f(x)$보다 적은 오차를 가짐이 증명되었다.

$$
numerical\ differentiation\ f'(x) = {{f(x+h)-f(x-h)}\over{2h}}
$$

그럼 이제 이러한 수치 미분을 함수로 구현해보겠다. 해당 함수의 인수는 총 세 개로, 각각 미분의 대상이 될 함수, 미분을 계산하는 변수, 그리고 아주 작은 수치값(이하 epsilon)이다. 여기서 세 번째 인수의 기본값은 1e-4로 하겠다.

```python
# steps/step04.py
def numerical_diff(f, x, eps=1e-4):
    x0 = Variable(x.data - eps) 
    x1 = Variable(x.data + eps) 
    y0 = f(x0) # f(x-h)
    y1 = f(x1) # f(x+h)
    return (y1.data - y0.data) / (2*eps)
```

실제 데이터는 Variable의 인스턴스 변수 data에 들어있다는 점만 조심하면 문제가 될 부분은 없다.

이제 앞서 구현한 Square 클래스를 대상으로 미분해보자.

```python
f = Square()
x = Variable(np.array(2.0))
dy = numerical_diff(f, x)
print(dy)
```

위를 실행하면 4.000000000004라는 값이 나오는데, 이는 $y=x^2$에서 $x=2$일 때의 미분값 4.0과 거의 유사함을 알 수 있다. 

### 4.3 합성 함수의 미분

앞서 수행한 계산은 간단한 함수에 대한 미분이었다. 이번엔 좀더 복잡한 합성 함수의 미분을 수행해보자.

```python
def complex_f(x):
    A = Square()
    B = Exp()
    C = Square()
    return C(B(A(x)))
```

```python
x = Variable(np.array(0.5))
dy = numerical_diff(complex_f, x)
print(dy)
```

여기서는 합성 함수 $y = (e^{x^2})^2$을 complex_f라는 함수로 따로 정의하였다. 파이썬에서는 함수도 객체이기 때문에 다른 함수에 인수로 전달해줄 수 있기 때문이다. 이러한 방식을 사용하면 복잡하게 조합된 다른 함수도 자동으로 미분할 수 있다.

### 4.4 수치 미분의 문제점

하지만 이러한 수치 미분에도 문제점이 존재한다. 
- 오차가 존재한다
- 계산량이 많다

여기서의 오차는 비록 매우 작은 값이지만, 경우에 따라 오차가 매우 커질 수도 있다. 이는 자릿수 누락에 의한 경우가 많으며 때론 치명적인 오류를 일으킬 수도 있다.

또한 변수가 여러 개인 경우 변수 각각을 미분해야 하므로 보통 수백만 개의 매개변수를 사용하는 신경망에서는 적용하기 어렵다.

이러한 문제점을 해결하기 위해 등장한 것이 바로 '역전파(Back-propagation)'이다. 이는 다음 step에서 설명하도록 하겠다.

\* 역전파가 있다고 해서 수치 미분이 버려지는 것은 아니다. 구현이 쉬운 수치 미분에 비해 역전파는 구현이 어렵고 복잡해 버그가 생길 우려가 높다. 따라서 역전파가 정확히 구현되었는지 확인을 하기 위해 수치 미분의 결과를 활용하기도 하는데, 이러한 방식을 '기울기 확인(gradient checking)' 과정이라고 한다.